# 케라스 RNN 신경망

SimpleRNN, LSTM, GRU 등이 구성되어있음

### 입력형식
LSTM(3, input_dim=1) 

3 : 메모리 셀의 개수(Dense에서 출력 뉴런수와 같은 의미)

input_dim : 입력 속성의 개수

input_length : 시퀀스 데이터의 입력 길이 

return_sequences : 시퀀스 출력 여부(True/False)

many to many : return_sequences = True

many to one : return_sequences = False

stateful : 상태유지 모드

현재 샘플의 마지막 셀의 상태가 다음 샘플의 입력으로 전달 할 것인지 여부를 지정

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler

In [2]:
path = r"Q:\Python_Data\dataset\samsung_stock_price.csv"
priceData = pd.read_csv(path, encoding='ANSI')

### 데이터 전처리

In [3]:
def MinMaxScaler(data):
    return (data-data.min(axis=0))/(data.max(axis=0)-data.min(axis=0)+1e-9)
 

In [4]:
data = np.loadtxt(r"Q:\Python_Data\dataset\data-02-stock_daily.csv", delimiter=",")
data = data[::-1] # 데이터 역순으로 옛날에서 최신으로

In [5]:
TEST_SIZE = 0.3
seqLength = 7 # window size : 7일 단위로 train 시키겠다.
dataDim = 5 # 시가, 종가, open 등 columns수 feature 수
hiddenDim = 10 # 각 셀에서 10개씩 출력
outputDim = 1 
lr = 0.01
iterations = 500 # 500회 반복

In [6]:
testSize = int(len(data) * TEST_SIZE)
trainSize = len(data) - testSize

trainSet = data[:trainSize]
testSet = data[trainSize-seqLength:]

In [7]:
trainSet = MinMaxScaler(trainSet)
testSet = MinMaxScaler(testSet)

### 모델 만들기

In [8]:
def buildDataSet(timeSeries, seqLength):
    xData, yData = [], []
    
    for i in range(len(timeSeries)-seqLength):
        tempX = timeSeries[i:i+seqLength,:]
        tempY = timeSeries[i+seqLength,[-1]] # 2차원 맞춰주기
        xData.append(tempX)
        yData.append(tempY)
    return np.array(xData), np.array(yData)

In [9]:
trnX, trnY = buildDataSet(trainSet, seqLength=7)
tstX, tstY = buildDataSet(testSet, 7)

In [10]:
print(trnX.shape, trnY.shape, )
print(tstX.shape, tstY.shape)

(506, 7, 5) (506, 1)
(219, 7, 5) (219, 1)


tensorflow

In [11]:
x = tf.placeholder(tf.float32, [None, seqLength, dataDim])
y = tf.placeholder(tf.float32, [None, 1])

In [12]:
cell = tf.contrib.rnn.BasicLSTMCell(
    num_units= hiddenDim, # 출력 Dimention 
    state_is_tuple=True,
    # 출력이 state와 output이 묶여서 나옴
    # True면 tuple형식 출력, False면 Tensor
    # ([1,2],[3,4])          [1,2,3,4]
    activation=tf.tanh
)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.


In [13]:
outputs, _states = tf.nn.dynamic_rnn(cell, x, dtype=tf.float32)
yhat = tf.contrib.layers.fully_connected(outputs[:,-1], outputDim)

Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Please use `layer.__call__` method instead.


In [14]:
loss = tf.reduce_mean(tf.square(yhat-y))
opt = tf.train.AdamOptimizer(lr)
train = opt.minimize(loss)

In [15]:
#rmse
targets=tf.placeholder(tf.float32, [None,1])
predictions=tf.placeholder(tf.float32, [None,1])
rmse=tf.sqrt(tf.reduce_mean(tf.square(targets-predictions)))

In [16]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(iterations):
        _,cv = sess.run([train, loss],feed_dict={x:trnX, y:tstY})
        print('step : {} loss : {}'.format(i, cv))
    testPredict = sess.run(yht, feed_dict={x:tstX, y:tstY})
    rmsev = sess.run(rmse, feed_dict={targets:tstY, predictions:testPredict})
    print('rmse : {}'.format(rmsev))

InternalError: 2 root error(s) found.
  (0) Internal: Blas GEMM launch failed : a.shape=(506, 15), b.shape=(15, 40), m=506, n=40, k=15
	 [[node rnn/while/basic_lstm_cell/MatMul (defined at C:\Users\chan\AppData\Roaming\Python\Python37\site-packages\tensorflow_core\python\framework\ops.py:1748) ]]
	 [[Mean/_13]]
  (1) Internal: Blas GEMM launch failed : a.shape=(506, 15), b.shape=(15, 40), m=506, n=40, k=15
	 [[node rnn/while/basic_lstm_cell/MatMul (defined at C:\Users\chan\AppData\Roaming\Python\Python37\site-packages\tensorflow_core\python\framework\ops.py:1748) ]]
0 successful operations.
0 derived errors ignored.

Original stack trace for 'rnn/while/basic_lstm_cell/MatMul':
  File "C:\Users\chan\anaconda3\envs\tf15_py37\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\chan\anaconda3\envs\tf15_py37\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\chan\anaconda3\envs\tf15_py37\lib\site-packages\ipykernel_launch